In [1]:
import pandas as pd 
import numpy as np


In [2]:
file = r'C:\A- internship project\Summer-internship-project\dataset.xlsx'

In [3]:
%pip install openpyxl

data = pd.read_excel(file)

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\tejag\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [4]:
data.head()

,km(constant),m/s,N,wt. %,x10-5 mm3/Nm,x10-5 mm3/Nm.1,x10-5 mm3/Nm.2
0,sliding distance,sliding velocity,normal load,filler %,SWR,SWR,SWR
1,5,2,20,4,2.627179,17.275695,6.766977
2,5,4,60,2,3.173835,0.887598,3.631082
3,5,4,60,0,0.109748,0.768239,4.225313
4,5,4,20,0,1.481603,0.65849,12.264384


In [5]:
data.tail()

,km(constant),m/s,N,wt. %,x10-5 mm3/Nm,x10-5 mm3/Nm.1,x10-5 mm3/Nm.2
23,5,4,40,2,4.276608,1.412087,7.100783
24,5,4,60,4,4.484781,6.846589,3.529443
25,5,2,20,0,0.411557,0.823113,6.667215
26,5,6,60,0,0.109748,0.713365,5.020989
27,5,6,40,2,0.847252,1.73485,4.559025


In [6]:
data.isnull().sum()

km(constant)      0
m/s               0
N                 0
wt. %             0
x10-5 mm3/Nm      0
x10-5 mm3/Nm.1    0
x10-5 mm3/Nm.2    0
dtype: int64

In [7]:
# Rename columns
data.rename(columns={
    "km(constant)": "Material_Constant",
    "m/s": "sliding velocity",
    "N": "normal load",
    "wt. %": "filler",
    "x10-5 mm3/Nm": "SWR1",
    "x10-5 mm3/Nm.1": "SWR2",
    "x10-5 mm3/Nm.2": "SWR3"
}, inplace=True)


In [8]:
data.head()

,Material_Constant,sliding velocity,normal load,filler,SWR1,SWR2,SWR3
0,sliding distance,sliding velocity,normal load,filler %,SWR,SWR,SWR
1,5,2,20,4,2.627179,17.275695,6.766977
2,5,4,60,2,3.173835,0.887598,3.631082
3,5,4,60,0,0.109748,0.768239,4.225313
4,5,4,20,0,1.481603,0.65849,12.264384


In [9]:
data = data.drop(0).reset_index(drop=True)


In [10]:
data.head()

,Material_Constant,sliding velocity,normal load,filler,SWR1,SWR2,SWR3
0,5,2,20,4,2.627179,17.275695,6.766977
1,5,4,60,2,3.173835,0.887598,3.631082
2,5,4,60,0,0.109748,0.768239,4.225313
3,5,4,20,0,1.481603,0.65849,12.264384
4,5,4,40,0,0.65849,0.740802,8.148819


In [11]:
x=data[["sliding velocity",	"normal load"	,"filler"]]
y=data["SWR1"]

In [12]:
x.head(3)

,sliding velocity,normal load,filler
0,2,20,4
1,4,60,2
2,4,60,0


In [13]:
y.head(3)

0    2.627179
1    3.173835
2    0.109748
Name: SWR1, dtype: object

In [14]:
import sklearn
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import  SVR
from sklearn.linear_model import  LinearRegression
from sklearn.tree import  DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [17]:
# Convert non-numeric columns to numeric (if any)
X = data.iloc[:, :-1].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN (from non-convertible strings)
X = X.dropna()
y = data.iloc[X.index, -1].astype(float)  # Ensure target is correctly aligned

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree Regressor
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f"Decision Tree Regression RMSE: {rmse:.4f}")


Decision Tree Regression RMSE: 1.7784


In [18]:
model2 = SVR()
model2.fit(X_train, y_train)

# Predict and evaluate
y_pred = model2.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f"SVR RMSE: {rmse:.4f}")


SVR RMSE: 0.8756


In [19]:
model3 = LinearRegression()
model3.fit(X_train, y_train)

# Predict and evaluate
y_pred = model3.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f" LinearRegression: {rmse:.4f}")


 LinearRegression: 1.9557


In [20]:
model4 =DecisionTreeRegressor()
model4.fit(X_train, y_train)

# Predict and evaluate
y_pred = model4.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f"DecisionTreeRegressor: {rmse:.4f}")


DecisionTreeRegressor: 1.7085


In [22]:
from xgboost import XGBRegressor


model5=XGBRegressor()
model5.fit(X_train, y_train)

# Predict and evaluate
y_pred = model5.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5  # Manually take the square root


print(f"DecisionTreeRegressor: {rmse:.4f}")

DecisionTreeRegressor: 1.6593


In [23]:
#ANN
# Build ANN model
import tensorflow as tf
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test))

# Evaluate model
loss, mae = model.evaluate(X_test, y_test)
print(f"ANN Model MAE: {mae:.4f}")

# Predict
y_pred = model.predict(X_test)

C:\Users\tejag\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - loss: 19.9916 - mae: 3.6415 - val_loss: 4.6693 - val_mae: 1.9023
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 22.6271 - mae: 3.9005 - val_loss: 6.7225 - val_mae: 2.2806
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 21.7515 - mae: 3.9163 - val_loss: 5.4283 - val_mae: 2.0199
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 17.6167 - mae: 3.4609 - val_loss: 2.9328 - val_mae: 1.4765
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 13.2565 - mae: 2.8582 - val_loss: 2.3085 - val_mae: 1.2764
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 13.3400 - mae: 2.7192 - val_loss: 2.1503 - val_mae: 1.2488
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 13.2841 - mae: 2.7197 - val_loss: 2.0566 - val_mae: 1.2155
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 17.0158 - mae: 2.9574 - val_loss: 1.8648 - val_mae: 1.1572
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 14.646